In [1]:
import tensorflow as tf
import scipy, numpy

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
# Configurações para o treinamento
train_data_dir = './data'
batch_size = 32 # 32 imagens por lote
num_epochs = 50 # 50 épocas de treinamento
image_size = (150, 150) # 150x150 pixels
num_classes = 2 # 2 classes: porco e gato

In [3]:
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    validation_split = 0.25, # 25% para validação
    shear_range = 0.2, # inclinação
    zoom_range = 0.2, # zoom
    horizontal_flip = True # espelhamento horizontal
)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'training'
)

Found 45 images belonging to 2 classes.


In [5]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'validation'
)

Found 13 images belonging to 2 classes.


In [6]:
model = Sequential() # instanciando o modelo
model.add(Conv2D(32, (3,3), input_shape=(image_size[0], image_size[1], 3), activation='relu')) # camada de convolução
model.add(MaxPooling2D(pool_size=(2,2))) # camada de pooling
model.add(Conv2D(64, (3,3), activation='relu')) # camada de convolução
model.add(MaxPooling2D(pool_size=(2,2))) # camada de pooling
model.add(Flatten()) # camada de achatamento ou vetorização
model.add(Dense(64, activation='relu')) # neurônios
model.add(Dropout(0.5)) # regularização 50% chance de desligar neurônios
model.add(Dense(1, activation='sigmoid')) # camada de saída (classificação binária)

In [7]:
# compilar o modelo
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy', # mais próximo de 0 melhor, 0.002 por exemplo
    metrics = ['accuracy']
)

In [8]:
# treinamento
model.fit(
    train_generator,
    epochs = num_epochs,
    validation_data = validation_generator
)

print('Treinamento concluído!')

# salvar o modelo
model.save('cfCleitonFerrugem.keras')

Epoch 1/50


2/2 [==============================] - 2s 480ms/step - loss: 1.5557 - accuracy: 0.4667 - val_loss: 0.6275 - val_accuracy: 0.6923
Epoch 2/50
2/2 [==============================] - 0s 335ms/step - loss: 0.8663 - accuracy: 0.4667 - val_loss: 0.6373 - val_accuracy: 0.6154
Epoch 3/50
2/2 [==============================] - 0s 171ms/step - loss: 0.9448 - accuracy: 0.4444 - val_loss: 0.6309 - val_accuracy: 0.6923
Epoch 4/50
2/2 [==============================] - 0s 311ms/step - loss: 0.6494 - accuracy: 0.6000 - val_loss: 0.5806 - val_accuracy: 0.7692
Epoch 5/50
2/2 [==============================] - 0s 291ms/step - loss: 0.7048 - accuracy: 0.5111 - val_loss: 0.6384 - val_accuracy: 0.4615
Epoch 6/50
2/2 [==============================] - 0s 162ms/step - loss: 0.6801 - accuracy: 0.5333 - val_loss: 0.5543 - val_accuracy: 0.9231
Epoch 7/50
2/2 [==============================] - 0s 158ms/step - loss: 0.6147 - accuracy: 0.6444 - val_loss: 0.5102 - val_accuracy: 0.8462
Epoch 8/50
2/2 [==